<a href="https://colab.research.google.com/github/Jignacio14/7506R-1C2023-GRUPO09/blob/Pruebas/7506R_TP1_GRUPO9_CHP1_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pruebas

En este jupyter notebook vamos a explorar un conjunto de datos sobre reservas de hoteles y tratar de hallar un modelo que nos permita predecir si la reserva va a ser cancelada 

Importamos todas las librerias que vamos a usar

In [17]:
import pandas as pd 
import numpy as np
import sklearn as sk
import seaborn as sns
from matplotlib import pyplot as plt

Cargamos los datos en un dataframe de pandas. Creamos una copia del dataframe original y trabajamos encima de la copia

In [18]:
hotelsDfOriginal = pd.read_csv("./hotels_train.csv")
hotelsdf = hotelsDfOriginal.copy()

print("El data frame esta compuesto por "f"{hotelsdf.shape[0]}"" filas y "f"{hotelsdf.shape[1]}"" columnas")

El data frame esta compuesto por 61913 filas y 33 columnas


Un vistaso básico a la información contenida en el dataframe:

In [19]:
pd.concat([hotelsdf.head(2), hotelsdf.sample(5), hotelsdf.tail(2)])

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,id,is_canceled
0,City Hotel,49,2016,September,37,5,1,2,1,0.0,...,NaN,0,Transient,115.50,0,1,Canceled,2016-08-25,7aa4cc6b-b92c-4061-b21d-4f9e0cac4689,1
1,Resort Hotel,4,2015,October,44,31,0,1,2,0.0,...,NaN,0,Transient,42.00,1,0,Check-Out,2015-11-01,b1428f80-c56c-4ae4-91a9-6962edae08b1,0
52698,City Hotel,49,2016,July,29,11,1,0,2,0.0,...,NaN,0,Transient,93.15,0,0,Canceled,2016-06-20,c45bf937-566f-4e68-b302-604cba26b402,1
28135,City Hotel,95,2016,November,45,5,0,1,1,0.0,...,NaN,0,Transient-Party,75.00,0,0,Canceled,2016-10-21,b0e5b157-80f7-4cf1-8924-a57b55378df9,1
41556,Resort Hotel,153,2015,August,33,9,2,3,2,0.0,...,NaN,0,Transient,236.00,1,1,Check-Out,2015-08-14,a18ecdbd-a31a-416f-9857-210733c5fda0,0
14613,City Hotel,44,2016,November,48,22,0,3,2,0.0,...,NaN,0,Transient,82.13,0,0,Canceled,2016-10-09,98a0421d-87bb-4285-bd24-6ac95eb0f06c,1
43191,Resort Hotel,1,2016,May,20,10,0,1,1,0.0,...,NaN,0,Transient-Party,109.00,0,1,Check-Out,2016-05-11,b55ace4b-c7d0-4f1f-bd70-00c64ed14447,0
61911,Resort Hotel,143,2015,October,41,10,2,5,2,0.0,...,NaN,0,Transient,36.05,0,0,Canceled,2015-07-08,01906859-1094-4e37-b13e-b015b594d159,1
61912,City Hotel,57,2016,April,17,17,2,5,2,0.0,...,NaN,0,Transient,143.29,0,0,Canceled,2016-04-09,7137e599-4442-41fb-b147-4aba9f344e38,1


Renombramos las columnas del dataframe con nombres mas claros y en español

In [20]:
nuevas_columnas = {'hotel':'hotel_name', #
    'lead_time':'lead_time',
    'arrival_date_year':'arrival_year',#
    'arrival_date_month':'arrival_month',#
    'arrival_date_week_number':'arrival_week_number',
    'arrival_date_day_of_month':'arrival_month_day',#
    'stays_in_weekend_nights':'weekend_nights_num',#
    'stays_in_week_nights':'week_nights_num', # 
    'adults':'adult_num',#
    'children':'children_num', #En el paper, dice que children es un int. Pero figura como float64. #TODO: Investigar
    'babies':'babies_num',
    'company':'ID_compania',
    'days_in_waiting_list': 'cantidad_dias_listaEspera',
    'customer_type': 'tipo_cliente',
    'adr': 'promedio_gastos_diarios', #No me termina de cerrar este valor
    'required_car_parking_spaces': 'cantidad_estacionamientos_pedidos',
    'total_of_special_requests': 'cantidad_pedidos_especiales',
    'reservation_status': 'estado_reserva',
    'reservation_status_date': 'estado_reserva_fecha',
    'id': 'id',
    'is_canceled': 'esta_cancelado',}

nuevas_columnas = {
    'adr':'average_daily_rate',
    'adults':'adult_num',
    'agent':'agent_id',
    'arrival_date_day_of_month':'arrival_month_day',
    'arrival_date_month':'arrival_month',
    'arrival_date_week_number':'arrival_week_number',
    'arrival_date_year':'arrival_year',
    'assigned_room_type':'assigned_room_type',
    'babies':'babies_num',
    'booking_changes':'booking_changes_num',
    'children':'children_num',
    'company':'company_id',
    'country':'country',
    'customer_type':'customer_type',
    'days_in_waiting_list':'days_in_waiting_list',
    'deposit_type':'deposit_type',
    'distribution_channel':'distribution_channel',
    'hotel':'hotel_name',
    'id':'booking_id', #chekear con el profesor
    'is_canceled':'is_canceled',
    'is_repeated_guest':'is_repeated_guest',
    'lead_time':'lead_time',
    'market_segment':'market_segment_type',
    'meal':'meal_type',
    'previous_bookings_not_canceled':'previous_bookings_not_canceled_num',
    'previous_cancellations':'previous_cancellations_num',
    'required_car_parking_spaces':'required_car_parking_spaces_num',
    'reservation_status':'reservation_status',
    'reservation_status_date':'reservation_status_date',
    'reserved_room_type':'reserved_room_type',
    'stays_in_weekend_nights':'weekend_nights_num',
    'stays_in_week_nights':'week_nights_num',
    'total_of_special_requests':'special_requests_num',
}

hotelsdf.rename(columns = nuevas_columnas, inplace = True)

In [21]:
#pd.concat([hotelsdf.head(2), hotelsdf.sample(5), hotelsdf.tail(2)])

Por otro lado, podemos observar que tipo de dato almacena cada columna y cuales tienen datos faltantes

In [22]:
hotelsdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61913 entries, 0 to 61912
Data columns (total 33 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   hotel_name                          61913 non-null  object 
 1   lead_time                           61913 non-null  int64  
 2   arrival_year                        61913 non-null  int64  
 3   arrival_month                       61913 non-null  object 
 4   arrival_week_number                 61913 non-null  int64  
 5   arrival_month_day                   61913 non-null  int64  
 6   weekend_nights_num                  61913 non-null  int64  
 7   week_nights_num                     61913 non-null  int64  
 8   adult_num                           61913 non-null  int64  
 9   children_num                        61909 non-null  float64
 10  babies_num                          61913 non-null  int64  
 11  meal_type                           61913

Segun la clasificacion de las variables podemos establecer lo siguiente sobre los datos extraidos del dataframe

Variables cuantitativas, entre las cuales podemos encontrar:

- lead_time "time beetwen reservation and arrival"
- arrival_date_year
- arrival_date_week_number  
- arrival_date_day_of_month
- stays_in_weekend_nights
- days_in_waiting_list
- stays_in_week_nights
- adult
- children
- babies
- previous_cancellations
- previous_bookings_not_canceled
- booking_changes
- days_in_waiting_list
- adr "average day rate"
- required_car_parking_spaces
- total_of_special_requests
- reservation_status_date


Creamos una lista con todas las variables cuantitativas

In [23]:
cuantitativas = ["lead_time"
,"arrival_date_year"
,"arrival_date_week_number"
,"arrival_date_day_of_month"
,"stays_in_weekend_nights"
,"days_in_waiting_list"
,"stays_in_week_nights"
,"adults"
,"children"
,"babies"
,"previous_cancellations"
,"previous_bookings_not_canceled"
,"booking_changes"
,"adr"
,"required_car_parking_spaces"
,"total_of_special_requests"
,"reservation_status_date"]

correlaciones = hotelsdf[cuantitativas].corr()

sns.set(style = 'darkgrid')
plt.figure( figsize = (12, 9))
sns.heatmap(data = correlaciones,annot = True, vmin = -1, vmax =1, fmt='.2f')
sns.color_palette("mako", as_cmap=True)
plt.show()

KeyError: ignored

In [ ]:
hotelsdf[cuantitativas].describe()

In [ ]:
hotelsdf.isna().sum()

Variables cualitativas

In [ ]:
cualitativas = ['hotel',"arrival_date_month", 'meal', 'country', 'market_segment', 'distribution_channel', 'is_repeated_guest', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type', 'reservation_status']

#no tiene sentido imprimir cosas como id, company, deposite_type

for variable in cualitativas:
  print(hotelsdf[variable].value_counts().index.tolist())



In [ ]:
print(pd.__version__)